In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import ast

In [2]:
link_list = list(pd.read_csv("tatCaTab_link.csv")["0"])

In [3]:
driver = webdriver.Chrome()

try:
    df = pd.read_csv('link_detail.csv')
    if df.empty:
        print("CSV file is empty")
        # Optionally create an empty DataFrame with your expected columns
        df = pd.DataFrame(columns=['link', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan'])
except pd.errors.EmptyDataError:
    print("File is completely empty")
    df = pd.DataFrame(columns=['link', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan'])
    df.to_csv('link_detail.csv', index=False)

for i in range(len(link_list)):
    # Go to detail page link of each question
    if link_list[i] in list(df['link'].values):
        continue

    driver.get(link_list[i])
    wait = WebDriverWait(driver, 3)

    try:
        # Try to find the readMore element
        wait.until(EC.presence_of_element_located((By.ID, "readLess")))
        read_more = driver.find_element(By.ID, "readMore")
        # Click readMore
        read_more.click()
    except:
        pass

    try:
        title_element = driver.find_element(By.CLASS_NAME, "main-title-sub")
        question_text = title_element.text.strip()
    except NoSuchElementException:
        question_text = ""

    try:
        article_div = driver.find_element(By.CLASS_NAME, "article")
        p_elements = article_div.find_elements(By.TAG_NAME, "p")
        answer_text = "\n".join([p.text.strip() for p in p_elements[1:] if p.text.strip() != ""])
    except NoSuchElementException:
        answer_text = ""

    try:
        related_links_section = driver.find_element(By.CLASS_NAME, "list-document")
        link_elements = related_links_section.find_elements(By.TAG_NAME, "a")

        links_related = []
        for link in link_elements:
            href = link.get_attribute("href")
            text = link.text.strip()
            if href:
                links_related.append(href)
    except:
        links_related = []

    try:
        question_links_section = driver.find_element(By.ID, "list-questions")
        link_elements = question_links_section.find_elements(By.TAG_NAME, "a")

        links_question = []
        for link in link_elements:
            href = link.get_attribute("href")
            text = link.text.strip()
            if href:
                links_question.append(href)
    except:
        links_question = []

    df.loc[i, ['link', 'cauHoi', 'cauTraLoi', 'TTHCLienQuan', 'cauHoiLienQuan']] = [link_list[i],
                                                                                    question_text, 
                                                                                    answer_text,
                                                                                    links_related,
                                                                                    links_question]
    df.to_csv("link_detail.csv", index=False)

    print(f"Done of {len(link_list)}: {i+1}", end='\r')

driver.quit()

In [4]:
df = pd.read_csv('link_detail.csv', index_col=False)
df.tail()

,link,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
9447,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Ủy ban nhân dân thành phố Huế muốn tổ chức họp...,Theo điểm b khoản 3 Điều 41 Luật báo chí năm 2...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...
9448,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Ủy viên Bộ Chính trị, Ban Bí thư thì ở Biệt th...",Theo quy định của Quyết định số 27/2015/QĐ-TTg...,[],[]
9449,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,“Cụt hoặc mất chức năng 01 bàn tay hoặc 01 bàn...,cần đến cơ sở y tế hoặc Trung tâm giám định y ...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...,['https://dichvucong.gov.vn/p/home/dvc-chi-tie...
9450,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"$questionInfo[""CONTENT""]","$i[""CONTENT""]",[],[]
9451,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"$questionInfo[""CONTENT""]","$i[""CONTENT""]",[],[]


In [5]:
df.TTHCLienQuan.apply(ast.literal_eval).explode().dropna().drop_duplicates().to_frame().reset_index(drop=True).to_csv("tthc_link.csv")